##This workbook describes a multi-position, z-stack acquisition with intermittent image processing
The specific example demonstrated incorporates a processing step to assess cell confluency levels  
(thanks to Manuel Leonetti, Keith Cheveralls @ czbiohub)

In [39]:
from py4j.java_gateway import JavaGateway, GatewayParameters
import numpy as np
import os
from skimage import filters


## Before running the next cell, be sure you've clicked "Create Python Bridge" in the mm2python plugin first.


In [41]:
# connect to org.mm2python
gateway = JavaGateway(gateway_parameters=GatewayParameters(auto_field=True))

# link to micro-manager control
gate = gateway
ep = gateway.entry_point
mm = ep.getStudio()
mmc = ep.getCMMCore()


## For precise control of micro-manager hardware, please see the documentation here:
Studio (mm)
Controls the GUI: Multidimensional-acquisition, metadata, display and high-level hardware control
- https://valelab4.ucsf.edu/~MM/doc-2.0.0-beta/mmstudio/org/micromanager/Studio.html

Core (mmc)
Controls the CORE: direct communication with hardware device adapters
- https://valelab4.ucsf.edu/~MM/doc-2.0.0-beta/mmcorej/mmcorej/CMMCore.html


In [29]:
# Setup your channel names and other hardawre parameters

Channels = [
    "EMCCD_Confocal40_DAPI",
    "EMCCD_Confocal40_GFP"
]

Lasers = [
    "Laser 405-Power Setpoint",
    "Laser 488-Power Setpoint"
]

LaserPowers = [
    10,
    10
]

Exposures = [ # Must be double
    50.0,
    50.0
]

Gain = [ # Must be double
    400.0,
    400.0
]

ZStack = [
    -6.0,    # Z-stack begin, relative
    16.0,   # Z-stack end, relative
    0.2     # Z-stack step
]

### important note:
If py4j complains that "method is not found" or similar, it could mean that you are passing the wrong data type.
for example:
	mmc.setExposure(100)	  # will return an error
	mmc.setExposure(100.0)	  # is ok!
python doesn't care about this difference, but java (and thus, micro-manager) does care!

In [31]:
# rename a few devices to make coding a little easier
# adjust the strings according to your hardware

zdevice = "PiezoZ"
xydevice = "XYStage"
config_group = "Channels-EMCCD"
camera = "Andor EMCCD"
laser_line = "Andor ILE-A"

### micro-manager 2.0 uses a concept of "datastores" to manage acquired data.
see documentation here for more info:
- https://micro-manager.org/wiki/Version_2.0_API


In [34]:
# to create a datastore, we need to define a save path:

# Save parameters
autoSave_path = "C:/path_for_your_data/"
prefix = "filename_prefix_here_"
num = 0

# check if the file already exists
# if it exists, increment num until we have a new file
while os.path.isdir(autoSave_path + prefix + str(num)):
    num += 1
fullpath = autoSave_path + prefix + str(num)

# finally, create the datastore!
autosavestore = mm.data().createMultipageTIFFDatastore(fullpath, True, True)
mm.displays().createDisplay(autosavestore)

JavaObject id=o5

In [36]:
# let's set up autofocus using the GUI's AutofocusManager
# comment this cell out if you are not using Autofocus!

# Set autofocus
af_manager = mm.getAutofocusManager()

# AFC is Leica's hardware autofocus device.  Rename this appropriately
af_manager.setAutofocusMethodByName("Adaptive Focus Control")
af_plugin = af_manager.getAutofocusMethod()

In [42]:
# optionally, you can assign devices to "ImageSynchro", 
# which requires all devices to settle before continuing acquisition
mmc.assignImageSynchro(zdevice)
mmc.assignImageSynchro(xydevice)
mmc.assignImageSynchro(mmc.getShutterDevice())
mmc.assignImageSynchro(mmc.getCameraDevice())
mmc.setAutoShutter(True)

### There are two ways one can snap an image in micro-manager:

1) using the Studio's SnapLiveManager  
2) using the Core's "snapImage" method

The Studio's SnapLiveManager opens a special window named "Snap/Live", takes a snap using current hardware settings
and places the image in that window

The Core's "snapImage" method will take a snap using current hardware settings, but does nothing else.  
It is up to the user to assign this data to a datastore.
  
See examples in the next cell

In [ ]:
import time


# the camera exposure takes some number of ms to finish.  We have to wait for that to finish
# or else "ep.getLastMeta()" will return None
def wait_for_meta(ep):
    ct = 0
    meta = ep.getLastMeta()
    while not meta:
        time.sleep(0.0001)
        ct += 1
        meta = ep.getLastMeta()
        if ct >= 10000:
            raise FileExistsError("timeout waiting for file exists")
    return meta


# Snap using SnapLiveManager
# Retrieve data using mm2python's memory mapped file system
def get_snap_data(mm, gate):
    
    # clear out old data from earlier snaps
    gate.entry_point.clearQueue()
    
    # Snaps and writes to snap/live view
    mm.live().snap(True)

    # Retrieve data from memory mapped files, np.memmap is functionally same as np.array
    meta = wait_for_meta(gate.entry_point)
    dat = np.memmap(meta.getFilepath(), dtype="uint16", mode='r+', offset=0,
                    shape=(meta.getxRange(), meta.getyRange()))

    return dat


# Snap using Core
# This pattern REQUIRES you to define a 
def get_snap_core(gate, datastore):
    mm = gate.entry_point.getStudio()
    mmc = gate.entry_point.getCMMCore()
    gate.entry_point.clearQueue()
    
    mmc.snapImage()
    tmp1 = mmc.getTaggedImage()
    
    # Convert tagged image into an Image object
    channel0 = mm.data().convertTaggedImage(tmp1)
    
    # assign metadata (coordinates) to this Image object
    channel0 = channel0.copyWith(
        channel0.getCoords().copy().channel(c).z(z).stagePosition(p).build(),
        channel0.getMetadata().copy().positionName("" + str(p)).build())
    
    # place this image (with its metadata) in a datastore
    # if this datastore is displayed in a window, mm2python will automatically recognize the new data
    datastore.putImage(channel0)
    
    # retrieve the data for python use
    # if datastore is not displayed, mm2python won't see new data, then function returns None
    try:
        meta = wait_for_meta(gate.entry_point)
    except FileExistsError:
        return None
    
    dat = np.memmap(meta.getFilepath(), dtype="uint16", mode='r+', offset=0,
                    shape=(meta.getxRange(), meta.getyRange()))

    return dat


### Next we'll define some processing functions to insert into our acquisition
The "overall_confluency" takes an image region and uses scikit image's filter library to asses cell confluency  
The "spread_test" splits the image into sub-regions, then passes those to "overall_confluency"  

"move_z" and "move_z_relative" are convenience abstractions


In [ ]:
# Confluency check for image within user-given thresholds
def overall_confluency(mid_image, total_pixels, lower_confluence_threshold, upper_confluence_threshold):
    # Using only middle slice, apply Gaussian filter
    filtered_mid_image = filters.gaussian(mid_image)

    # Threshold between background and non-background using Li Thresholding
    val = filters.threshold_li(filtered_mid_image)

    # Compute percentage of non-background pixels
    blue_pixels = len(filtered_mid_image[filtered_mid_image >= val])
    blue_pixel_percentage = (blue_pixels / total_pixels) * 100.0
    # print("Blue pixels:", str(blue_pixels))
    # print("Confluency:", str(blue_pixel_percentage))

    if blue_pixel_percentage >= lower_confluence_threshold and \
            blue_pixel_percentage <= upper_confluence_threshold:
        return True
    else:
        return False


# Test for good confluency overall and spread of cells
def spread_test(slice):
    # Set thresholds for making confluency decisions
    global_lower_confluence_threshold = 15
    global_upper_confluence_threshold = 46
    tile_lower_confluence_threshold = 10
    tile_upper_confluence_threshold = 50
    total_image_pixels = float(1024.0 * 1024.0)
    slice_factor = 256
    sub_image_pixels = float(slice_factor * slice_factor)
    false_counter = 0

    if overall_confluency(slice, total_image_pixels, global_lower_confluence_threshold, global_upper_confluence_threshold):
        addition_x = 0
        addition_y = 0

        # Splitting image into 16 256 x 256 tiles and testing confluency on each tile
        for n in range(16):
            sub_image = np.empty((slice_factor, slice_factor))
            for i in range(slice_factor):
                for j in range(slice_factor):
                    sub_image[i][j] = slice[i + addition_x][j + addition_y]

            # Actual confluency check for single tile
            sub_image_confluency = overall_confluency(sub_image, sub_image_pixels, tile_lower_confluence_threshold, tile_upper_confluence_threshold)
            if sub_image_confluency == False:
                false_counter += 1

            # Updates values for tiling
            if n == 3 or n == 7 or n == 11:
                addition_y += slice_factor
                addition_x = 0
            else:
                addition_x += slice_factor

        if false_counter > 4:
            print("Bad spread")
            return False
        else:
            return True
    else:
        print("Overall confluency fail")
        return False
    

# Quick command to move FocusDrive to absolute position
def move_z(mmc, zdevice, newZ):
    mmc.setPosition(zdevice, newZ)
    mmc.waitForDevice(zdevice)
    curPos = mmc.getPosition(zdevice)
    return curPos


# Quick command to move FocusDrive relative to the current position
def move_z_relative(mmc, zdevice, offset):
    mmc.setRelativePosition(zdevice, offset)
    mmc.waitForDevice(zdevice)
    curPos = mmc.getPosition(zdevice)
    return curPos


### Setup is done, now we can run the acquisition loop

In [ ]:
pl = mm.getPositionList()

# Position loop
for p in range(pl.getNumberOfPositions()):
    
    # Reset PiezoZ
    mmc.setPosition(zdevice, 0.0)

    # Move to next position
    nextPosition = pl.getPosition(p)
    nextPosition.goToPosition(nextPosition, mmc)
    
    # Autofocus
    print("Focusing...")
    mmc.setConfig(config_group, Channels[0])
    mmc.waitForConfig(config_group, Channels[0])
    mmc.setExposure(float(Exposures[0]))
    mmc.setProperty(laser_line, Lasers[0], LaserPowers[0])
    mmc.setProperty(camera, "Gain", Gain[0])
    try:
        af_plugin.fullFocus()
    except:
        print("AFC FAIL")
    mmc.waitForSystem()
    focalPlane = mmc.getPosition(zdevice)
    curPos = focalPlane
    print("Found focus.")
    mmc.waitForSystem()

    # spread_test is a function that evaluates cell density (confluency)
    if spread_test(get_snap_data(mm, gate)):
        print("Good confluency")

        # Move to bottom of stack
        print("Moving the stage to: " + str(ZStack[0]))
        floor = move_z_relative(mmc, zdevice, ZStack[0])

        # Channel loop
        for c in range(len(Channels)):
            print("Now imaging:" + Channels[c])
            mmc.setConfig(config_group, Channels[c])
            mmc.setProperty(laser_line, Lasers[c], LaserPowers[c])
            mmc.setExposure(float(Exposures[c]))
            mmc.setProperty(camera, "Gain", Gain[c])
            z = 0
            curPos = mmc.getPosition(zdevice)

            # Z-position loop
            while curPos <= focalPlane + ZStack[1]:
                
                # acquire data
                mmc.waitForImageSynchro()
                get_snap_core(gate, autosavestore)
                
                # Move to next z-position
                curPos = move_z_relative(mmc, zdevice, ZStack[2])
                z += 1

            move_z(mmc, zdevice, floor)

        move_z(mmc, zdevice, focalPlane)

    else: # Confluency at this position is poor, moving to next position
        print("Poor confluency, skipping position.")

autosavestore.freeze()